In [ ]:
# !conda install keras
# !conda install  nltk -y

In [ ]:
# !conda list | grep keras

In [ ]:
import tensorflow

In [ ]:
from dateutil.parser import parse
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import re

import numpy as np
import pandas as pd

import keras
from keras.metrics import Mean

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model

from keras import backend as K



nltk.download('punkt')

plt.style.use('fivethirtyeight')

In [ ]:
import sys
sys.setrecursionlimit(10000)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# not_punctuation = r'[^\w\s]'

In [ ]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip').fillna(' ')
test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip').fillna(' ')

In [ ]:
train

In [ ]:
test

### 2. Data exploring









### 3. Data transforming

In [ ]:
def clean_comment(comment):

    comment = comment.lower()

    # replace common short expression
    comment = re.sub(r"\'ve", ' have', comment)
    comment = re.sub(r"\'s", " ", comment)
    comment = re.sub(r"can't", "cannot ", comment)
    comment = re.sub(r"n't", " not ", comment)
    comment = re.sub(r"i'm", "i am ", comment)
    comment = re.sub(r"\'re", " are ", comment)
    comment = re.sub(r"\'d", " would ", comment)
    comment = re.sub(r"\'ll", " will ", comment)
    comment = re.sub(r"\'scuse", " excuse ", comment)

    # remove stand-alone numbers
    comment = re.sub(r'(?<!\S)\d+(?!\S)', '', comment)

    # remove non-character
    comment = re.sub(r'[^\w\s]', ' ', comment)
    
    # remove spaces
    comment = re.sub(r'\s+', ' ', comment)
    
    return comment

In [ ]:
def build_stopwords():
    result = stopwords.words('english')
    for c in string.ascii_uppercase:
        result.append( c )
    for c in string.ascii_lowercase:
        result.append(c)
    for c in string.digits:
        result.append(c)
    return result
all_stopwords = build_stopwords()

In [ ]:
def remove_stopwords(comment):
    return ' '.join([word for word in comment.split() if word not in all_stopwords])

In [ ]:
def stemSentence(comment):
    porter = PorterStemmer()
    token_words=word_tokenize(comment)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
    return " ".join(stem_sentence)

In [ ]:
def lemmaSentence(comment):
    wordnet_lemmatizer = WordNetLemmatizer()
    token_words = nltk.word_tokenize(comment)
    lemmas = []
    for word in token_words:
        lemmas.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
    return " ".join(lemmas)

In [ ]:
train['comment_text'] = train['comment_text'].apply(lambda comment: clean_comment(comment))
test['comment_text'] = test['comment_text'].apply(lambda comment: clean_comment(comment))


In [ ]:
train['comment_text'] = train['comment_text'].apply(lambda comment: remove_stopwords(comment))
test['comment_text'] = test['comment_text'].apply(lambda comment: remove_stopwords(comment))

In [ ]:
train['comment_text'] = train['comment_text'].apply(lambda comment: lemmaSentence(comment))
test['comment_text'] = test['comment_text'].apply(lambda comment: lemmaSentence(comment))

In [ ]:
train['comment_text']


In [ ]:
test['comment_text']

In [ ]:

# vect = TfidfVectorizer(max_features=10000,stop_words= all_stopwords)
# vect

In [ ]:

all_comments = train['comment_text'].append(test['comment_text'])

In [ ]:

max_features = 20000
tokenizer = Tokenizer(num_words=max_features)

tokenizer.fit_on_texts(list(all_comments))

In [ ]:
train_tokenized_comment = tokenizer.texts_to_sequences(train['comment_text'])
test_tokenized_comment = tokenizer.texts_to_sequences(test['comment_text'])


In [ ]:
def len_tokenied_comment(comments):
    result = []
    for comment in comments:
        result.append(len(comment))
    return result

train_tokenized_comment_len = len_tokenied_comment(train_tokenized_comment)
test_tokenized_comment_len = len_tokenied_comment(test_tokenized_comment)

In [ ]:
print(np.quantile(train_tokenized_comment_len, q=.9))
plt.hist(train_tokenized_comment_len, bins = 200)
plt.show()

In [ ]:
train_tokenized_comment

In [ ]:
test_tokenized_comment

In [ ]:
max_token_len = 300
train_tokenized_comment = pad_sequences(train_tokenized_comment, maxlen=max_token_len)
test_tokenized_comment = pad_sequences(test_tokenized_comment, maxlen=max_token_len)

In [ ]:
train_tokenized_comment_len = len_tokenied_comment(train_tokenized_comment)
test_tokenized_comment_len = len_tokenied_comment(test_tokenized_comment)

print(np.quantile(train_tokenized_comment_len, q=.9))
plt.hist(train_tokenized_comment_len, bins = 200)
plt.show()

In [ ]:
inp = Input(shape=(max_token_len, ))

embed_size = 128
x = Embedding(max_features, embed_size)(inp)
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

In [ ]:
batch_size = 32
epochs = 2

In [ ]:
train_classify = train[class_names].values

model.fit(train_tokenized_comment, train_classify, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
import pickle

In [ ]:
filename = 'model_20201027.sav'

In [ ]:
pickle.dump(model, open(filename, 'wb'))

In [ ]:
model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)

In [ ]:
model.evaluate(train_tokenized_comment, train_classify)


In [ ]:
# for i,index in enumerate(train.index):
#     train[index,'tokenized_comment'] = train_tokenized_comment[i]

In [ ]:
%%time

test_predict = model.predict(test_tokenized_comment , batch_size=batch_size, verbose=1)

# predict_result = []
# for test_data in test_tokenized_comment:
#     test_data = np.array(test_data).T
#     predict_result.append(
#         loaded_model.predict(test_data)
#     )
    
# predict_result = loaded_model.predict_classes(train_tokenized_comment)

In [ ]:
test['comment_text']

In [ ]:
test_predict_df = pd.DataFrame(test_predict)
test_predict_df

In [ ]:
test[class_names] = test_predict_df
test

In [ ]:
submission = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')

In [ ]:
submission

In [ ]:
submission[class_names] = test_predict_df
submission

In [ ]:
submission.to_csv('submission.csv', index=False)